In [97]:
from groq import Groq
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

load_dotenv()

True

In [98]:
client = Groq(
    api_key=os.getenv("GROQ_API_KEY")
)
model_name = "llama-3.1-8b-instant"

In [99]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response

Get LLM Response

In [100]:
messages = [{"role": "user", "content": "what is the capital of India?"}]
response = get_chatbot_response(client, model_name, messages)

In [101]:
response

'The capital of India is New Delhi.'

# Prompt Engineering

STRUCTURED OUTPUT

In [102]:
system_prompt = """
You are a helpful assistant that answer the questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. You are not allowed anything to write other than the json object.
[
    {
        "country": the country that you will get the captial of
        "capital": the capital of the country stated
    }
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "what is the capital of India?"})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "India",
        "capital": "New Delhi"
    }
]


In [103]:
type(response)

str

In [104]:
json_response = json.loads(response)
json_response

[{'country': 'India', 'capital': 'New Delhi'}]

In [105]:
type(json_response[0]),json_response[0]['capital']

(dict, 'New Delhi')

# Input Structuring

In [106]:
user_input = """
Get me the capitals of the following countries:
```
1. India
2. Germany
3. France
```
"""

messages = [{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":user_input})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "India",
        "capital": "New Delhi"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [107]:
json_response = json.loads(response)
json_response

[{'country': 'India', 'capital': 'New Delhi'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

# Give the model time to think (Chain of Thought)

In [108]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structureed json format exactly like the one below. You are not allowed anything to write other than the json object.
{
    result: The final number resulted form calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 4
}


In [109]:
259/2*8654+91072*33-12971

4113098.0

In [110]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structureed json format exactly like the one below. You are not allowed anything to write other than the json object.
{
    result: The final number resulted form calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 17511191
}


In [111]:
4113098.0 - 17511191

-13398093.0

In [112]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structureed json format exactly like the one below. You are not allowed anything to write other than the json object.
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
    result: The final number resulted form calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "steps": "First, we need to follow the BEDMAS order of operations. 
    1. Divide 259 by 2: 259 / 2 = 129.5
    2. Multiply 129.5 by 8654: 129.5 * 8654 = 1121111
    3. Multiply 91072 by 33: 91072 * 33 = 3005016
    4. Add 1121111 and 3005016: 1121111 + 3005016 = 4126137
    5. Subtract 12971 from 4126137: 4126137 - 12971 = 4114026",
    "result": 4114026
}


In [113]:
4113098 - 4114026

-928

# RAG - Retrieval Augmented Generation

In [114]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

Since I'm not aware of any official information about the iPhone 16, I'll provide you with some general information about the latest iPhone models and some rumored features that might be included in future iPhone models.

However, I can tell you about the latest iPhone models, such as the iPhone 14 series, which includes:

1. iPhone 14
2. iPhone 14 Plus
3. iPhone 14 Pro
4. iPhone 14 Pro Max

Some of the notable features of the iPhone 14 series include:

1. Improved cameras with a new 48MP main camera on the Pro models
2. Faster A16 Bionic chip
3. Longer battery life
4. New colors and designs
5. Enhanced display with a higher refresh rate

As for the iPhone 16, there's no official information available yet. However, based on rumors and leaks, here are some potential features that might be included:

1. Improved cameras with a new periscope lens or a higher-resolution main camera
2. Faster A17 Bionic chip or a new chip design
3. Enhanced display with a higher refresh rate or a new OLED p

In [115]:
iphone_16 = """
Apple has officially launched the iPhone 16 series, introducing the most advanced iPhone lineup to date. The devices feature a redesigned internal architecture, larger batteries, and improved thermal management to sustain high performance during extended workloads. 

At the core, the iPhone 16 and iPhone 16 Pro models are powered by the new **A19 Bionic chip** (built on TSMC’s 3nm process), delivering significant improvements in CPU, GPU, and Neural Engine performance. The A19 Pro in the Pro models supports advanced AI workloads directly on-device, enabling real-time generative tasks and improved efficiency across machine learning applications. 

The display technology has been upgraded with a brighter **Super Retina XDR OLED** panel, supporting up to 2000 nits peak brightness outdoors, ProMotion with adaptive refresh rates up to 120 Hz, and Always-On Display. The Pro models also integrate **LTPO technology** for better power management. 

The camera system has been enhanced with a new **48 MP main sensor** with larger pixels, improved optical image stabilization, and upgraded computational photography pipelines. A dedicated **LiDAR+ sensor** enables faster depth mapping and AR experiences, while the Pro models add a **5x telephoto zoom** with enhanced image stabilization. Video capture now includes **8K ProRes recording** and next-generation “Cinematic HDR+.” 

Connectivity has been advanced with the full switch to **USB-C**, supporting USB 3 and Thunderbolt 3 on the Pro models for up to 40 Gbps transfer speeds. Wireless technologies include Wi-Fi 7, Bluetooth 5.4, ultra-wideband (UWB) Gen-2, and upgraded MagSafe with faster 25 W wireless charging. Reverse wireless charging for accessories like AirPods and Apple Watch has also been introduced.  

Other notable features include improved Ceramic Shield with higher drop resistance, IP68 water and dust resistance, and tighter integration with iOS 19 features such as **Apple Intelligence**, offering AI-driven text generation, image enhancement, and system-level assistance. Battery life is extended across all models with higher-density cells and improved power efficiency. 

Overall, the iPhone 16 marks a major leap in Apple’s smartphone technology, combining advanced hardware, cutting-edge connectivity, and new AI capabilities into a cohesive platform for developers, professionals, and consumers alike.
"""



In [116]:
user_prompt = """
{iphone_16}
What's new in iphone 16?
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

Since the iPhone 16 has not been officially released by Apple yet, I'll provide you with some rumored features and speculations based on industry trends and leaks. Keep in mind that these are not confirmed and may change before the actual release.

**Rumored Features:**

1. **Design:** The iPhone 16 is expected to have a similar design to the iPhone 15, with a stainless steel frame and a tougher glass back. However, some rumors suggest that it might have a slightly smaller notch or even a notchless design.
2. **Display:** The iPhone 16 is expected to feature a higher refresh rate display, possibly up to 120Hz or even 144Hz. This would provide a smoother and more responsive user experience.
3. **Camera:** The iPhone 16 is rumored to have a quad-camera setup, with a new periscope lens that would allow for even better zoom capabilities. There might also be improvements to the front camera, such as a higher resolution or a wider angle lens.
4. **Battery Life:** The iPhone 16 is expected to

### Automatically Extract context data from database.

In [117]:
samsung_s23 = """
Samsung officially launched the Galaxy S23 series, representing a refined evolution of its flagship smartphone lineup with a focus on performance, camera innovation, and sustainable design. The series includes the Galaxy S23, S23+, and S23 Ultra — each delivering premium build quality, top-tier hardware, and a host of intelligent software enhancements.

At the heart of the Galaxy S23 series is the Qualcomm Snapdragon 8 Gen 2 for Galaxy chipset, a custom-tuned version built on TSMC’s 4nm process. This processor offers enhanced CPU and GPU performance with improved energy efficiency and sustained thermal stability, enabling smooth multitasking, console-level gaming, and accelerated AI-driven features such as image optimization and voice recognition.

The display continues Samsung’s excellence with Dynamic AMOLED 2X panels, supporting QHD+ resolution (on the Ultra model), adaptive 120 Hz refresh rate, and Vision Booster technology that adjusts brightness and contrast in real time for visibility under bright sunlight. The screens also reach up to 1750 nits peak brightness, ensuring vibrant color reproduction and HDR10+ support across all models.

Camera technology takes a major step forward with the 200 MP ISOCELL HP2 sensor on the Galaxy S23 Ultra, delivering exceptional detail, low-light performance, and advanced noise reduction. The system supports Super HDR, 8K video recording at 30 fps, and enhanced Nightography for both photo and video. AI-driven image processing refines portrait mode, astrophotography, and object tracking, while the Expert RAW app gives professionals full manual control for RAW capture.

Connectivity features include 5G (mmWave and sub-6GHz), Wi-Fi 6E, Bluetooth 5.3, and USB-C with USB 3.2 Gen 1 transfer speeds. The S23 Ultra also supports UWB (Ultra-Wideband) for precise device tracking and ecosystem integration. Powering all models are larger, optimized batteries — up to 5000 mAh on the Ultra — with 45 W fast charging, 15 W wireless charging, and reverse wireless PowerShare for accessories.

Samsung emphasizes durability and sustainability with Gorilla Glass Victus 2, an Armor Aluminum frame, and the use of recycled materials throughout the device construction. The phones are IP68-rated for water and dust resistance.

Running One UI 5.1 based on Android 13, the Galaxy S23 series introduces advanced customization, productivity enhancements, and deep integration with the Samsung ecosystem, including Galaxy Buds, Galaxy Watch, and Windows PCs.

Overall, the Galaxy S23 series delivers a blend of raw performance, refined design, and intelligent camera capabilities, reinforcing Samsung’s leadership in premium Android smartphones.
"""

In [118]:
data = [iphone_16,samsung_s23]

In [119]:
user_prompt = """What's new in iphone 16?"""

In [120]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from groq import Groq
import os

# 1. Local Embeddings (384 dimensions)
embedding_client = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={
        'normalize_embeddings': True
    }
)

In [121]:
embedding_model_name = "BAAI/bge-small-en-v1.5"

In [122]:
groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

def get_embedding(embedding_client, embedding_model_name, text_input):
    """
    Get embeddings locally (384 dimensions with batch_size=32)
    
    Args:
        embedding_client: HuggingFaceBgeEmbeddings instance
        model_name: Model name (BAAI/bge-small-en-v1.5)
        text_input: Text or list of texts to embed
    
    Returns:
        List of 384-dimensional embeddings
    """
    if isinstance(text_input, str):
        text_input = [text_input]
    
    embeddings = []
    for text in text_input:
        embedding = embedding_client.embed_query(text)
        embeddings.append(embedding)
    
    return embeddings

In [123]:
user_prompt_embedding = get_embedding(embedding_client, model_name, user_prompt)[0]

In [124]:
data_embeddings = [get_embedding(embedding_client, embedding_model_name, text)[0] for text in data]

In [125]:
len(data_embeddings)

2

In [126]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embeddings)

In [127]:
data_similarity

array([[0.77290877, 0.52220754]])

In [128]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

0

In [129]:
data[closest_entry_index]

'\nApple has officially launched the iPhone 16 series, introducing the most advanced iPhone lineup to date. The devices feature a redesigned internal architecture, larger batteries, and improved thermal management to sustain high performance during extended workloads. \n\nAt the core, the iPhone 16 and iPhone 16 Pro models are powered by the new **A19 Bionic chip** (built on TSMC’s 3nm process), delivering significant improvements in CPU, GPU, and Neural Engine performance. The A19 Pro in the Pro models supports advanced AI workloads directly on-device, enabling real-time generative tasks and improved efficiency across machine learning applications. \n\nThe display technology has been upgraded with a brighter **Super Retina XDR OLED** panel, supporting up to 2000 nits peak brightness outdoors, ProMotion with adaptive refresh rates up to 120 Hz, and Always-On Display. The Pro models also integrate **LTPO technology** for better power management. \n\nThe camera system has been enhanced w

In [130]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [131]:
print(user_prompt_with_data)



Apple has officially launched the iPhone 16 series, introducing the most advanced iPhone lineup to date. The devices feature a redesigned internal architecture, larger batteries, and improved thermal management to sustain high performance during extended workloads. 

At the core, the iPhone 16 and iPhone 16 Pro models are powered by the new **A19 Bionic chip** (built on TSMC’s 3nm process), delivering significant improvements in CPU, GPU, and Neural Engine performance. The A19 Pro in the Pro models supports advanced AI workloads directly on-device, enabling real-time generative tasks and improved efficiency across machine learning applications. 

The display technology has been upgraded with a brighter **Super Retina XDR OLED** panel, supporting up to 2000 nits peak brightness outdoors, ProMotion with adaptive refresh rates up to 120 Hz, and Always-On Display. The Pro models also integrate **LTPO technology** for better power management. 

The camera system has been enhanced with a n

In [132]:
messages = [{"role":"user","content":user_prompt_with_data}]
response = get_chatbot_response(client, model_name, messages)
print(response)

According to the provided information, the following are the new features and upgrades in the iPhone 16 series:

1. **Redesigned internal architecture**: Larger batteries and improved thermal management for sustained high performance.
2. **A19 Bionic chip**: Built on TSMC's 3nm process, delivering significant improvements in CPU, GPU, and Neural Engine performance.
3. **A19 Pro chip**: Supports advanced AI workloads directly on-device, enabling real-time generative tasks and improved efficiency across machine learning applications.
4. **Super Retina XDR OLED display**: Brighter panel with up to 2000 nits peak brightness outdoors, ProMotion with adaptive refresh rates up to 120 Hz, and Always-On Display.
5. **LTPO technology**: Integrated in Pro models for better power management.
6. **48 MP main sensor**: Larger pixels, improved optical image stabilization, and upgraded computational photography pipelines.
7. **LiDAR+ sensor**: Enables faster depth mapping and AR experiences.
8. **5x t